In [77]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [109]:
city_data_df = pd.read_csv("cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,saldanha,-33.0117,17.9442,62.13,79,100,24.94,ZA,1637705348
1,1,iquique,-20.2208,-70.1431,64.44,71,98,4.56,CL,1637705348
2,2,alekseyevsk,57.8406,108.3492,13.86,98,100,2.08,RU,1637705348
3,3,kamaishi,39.2667,141.8833,41.49,93,79,6.20,JP,1637705348
4,4,dikson,73.5069,80.5464,1.13,94,100,12.73,RU,1637705349


In [110]:
gmaps.configure(api_key=g_key)

In [111]:
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [112]:
narrowed_city_df = city_data_df.loc[(city_data_df["Max Temp"] < 80) & (city_data_df["Max Temp"] > 70) \
                                    & (city_data_df["Wind Speed"] < 10) \
                                    & (city_data_df["Cloudiness"] == 0)].dropna()
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
52,52,mecca,21.4267,39.8261,73.35,50,0,3.4,SA,1637705196
365,365,moron,-34.6534,-58.6198,78.06,34,0,3.0,AR,1637705438
435,435,vila velha,-20.3297,-40.2925,75.15,85,0,4.0,BR,1637705456


In [113]:
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
52,mecca,SA,21.4267,39.8261,
365,moron,AR,-34.6534,-58.6198,
435,vila velha,BR,-20.3297,-40.2925,


In [114]:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    name_address = requests.get(base_url, params=params)
    
    name_address = name_address.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
52,mecca,SA,21.4267,39.8261,
365,moron,AR,-34.6534,-58.6198,
435,vila velha,BR,-20.3297,-40.2925,


In [115]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [116]:
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))